# 5. Image data types and what they mean

在 `skimage` 中，图像只是 numpy 数组，它支持多种数据类型，即“dtypes”。为了避免扭曲图像强度（请参阅 [Rescaling intensity values](https://scikit-image.org/docs/stable/user_guide/data_types.html#rescaling-intensity-values)），我们假设图像使用以下 dtype 范围：

|Data type|Range|
|:-|:-|
|uint8|0 to 255|
|uint16|0 to 65535|
|uint32|0 to 2<sup>32</sup> - 1|
|float|-1 to 1 or 0 to 1|
|int8|-128 to 127|
|int16|-32768 to 32767|
|int32|-2<sup>31</sup> to 2<sup>31</sup> - 1|

请注意，float 图像应限制在 -1 到 1 的范围内，即使数据类型本身可以超出此范围；另一方面，所有 integer 数据类型都具有可以跨越整个数据类型范围的像素强度。除少数例外，不支持 64-bit (u)int 图像。

`skimage` 中的函数经过设计，可以接受任何这些数据类型，但为了提高效率，可能会返回不同数据类型的图像（see [Output types](https://scikit-image.org/docs/stable/user_guide/data_types.html#output-types)）。如果您需要特定的数据类型，`skimage` 提供了转换数据类型并正确重新调整图像强度的实用程序函数（see [Input types](https://scikit-image.org/docs/stable/user_guide/data_types.html#input-types)）。您永远不应该在图像上使用 `astype`，因为它违反了有关 dtype 范围的以下假设：

In [2]:
from skimage.util import img_as_float
import numpy as np
image = np.arange(0, 50, 10, dtype=np.uint8)
print(image.astype(float)) # These float values are out of range.

[ 0. 10. 20. 30. 40.]


In [3]:
print(img_as_float(image))

[0.         0.03921569 0.07843137 0.11764706 0.15686275]


## 5.1. Input types 

尽管我们的目标是保留输入图像的数据范围和类型，但函数可能仅支持这些数据类型的子集。在这种情况下，输入将被转换为所需的类型（如果可能），并且如果需要内存副本，则会将警告消息打印到日志中。类型要求应在文档字符串中注明。

main package 中的以下实用函数可供开发人员和用户使用：

|Function name|Description|
|:-|:-|
|img_as_float|Convert to floating point (integer types become 64-bit floats)|
|img_as_ubyte|Convert to 8-bit uint.|
|img_as_uint|Convert to 16-bit uint.|
|img_as_int|Convert to 16-bit int.|

这些函数将图像转换为所需的数据类型并正确调整其值：

In [4]:
from skimage.util import img_as_ubyte
image = np.array([0, 0.5, 1], dtype=float)
img_as_ubyte(image)

array([  0, 128, 255], dtype=uint8)

当心！这些转换可能会导致精度损失，因为 8 bits 无法保存与 64 bits 相同的信息量：

In [6]:
image = np.array([0, 0.5, 0.503, 1], dtype=float)
img_as_ubyte(image)

array([  0, 128, 128, 255], dtype=uint8)

请注意，`img_as_float` 将保留浮点类型的精度，并且不会自动重新调整浮点输入的范围。

此外，某些函数采用 `preserve_range` 参数，其中范围转换很方便但不是必需的。例如，`transform.warp` 中的插值需要 float 类型的图像，其范围应为 [0, 1]。因此，默认情况下，输入图像将重新缩放到此范围。然而，在某些情况下，图像值表示用户不希望重新缩放的物理测量值，例如温度或降雨量值。使用`preserve_range=True`，即使输出是浮点图像，也将保留数据的原始范围。然后，用户必须确保下游函数正确处理该非标准图像，下游函数可能期望图像位于 [0, 1] 中。一般来说，除非函数具有 `preserve_range=False` 关键字参数，否则浮点输入不会自动重新缩放。

In [7]:
from skimage import data
from skimage.transform import rescale
image = data.coins()
image.dtype, image.min(), image.max(), image.shape

(dtype('uint8'), 1, 252, (303, 384))

In [8]:
rescaled = rescale(image, 0.5)
(rescaled.dtype, np.round(rescaled.min(), 4),
 np.round(rescaled.max(), 4), rescaled.shape)

(dtype('float64'), 0.0189, 0.9156, (152, 192))

In [10]:
rescaled = rescale(image, 0.5, preserve_range=True)
(rescaled.dtype, np.round(rescaled.min()),
 np.round(rescaled.max()), rescaled.shape)

(dtype('float64'), 5.0, 233.0, (152, 192))

## 5.2. Output types

函数的输出类型由函数作者确定，并为了用户的利益而进行记录。虽然这要求用户将输出显式转换为所需的格式，但它确保不会发生不必要的数据复制。

需要特定类型输出（例如，出于显示目的）的用户可以编写：

In [ ]:
from skimage.util import img_as_uint
out = img_as_uint(sobel(image))
plt.imshow(out)

## 5.3. Working with OpenCV

您可能需要使用通过 `skimage` 和 [OpenCV](https://opencv.org/) 创建的图像，反之亦然。OpenCV 图像数据可以在 NumPy（以及 scikit-image）中访问（无需复制）。OpenCV 对彩色图像使用 BGR（而不是 scikit-image 的 RGB），其 dtype 默认为 uint8（See [Image data types and what they mean](https://scikit-image.org/docs/stable/user_guide/data_types.html#image-data-types-and-what-they-mean)）。BGR 代表蓝绿红。

### 5.3.1. Converting BGR to RGB or vice versa 

`skimage` 和 `OpenCV` 中的彩色图像有 3 个维度：宽度、高度和颜色。RGB 和 BGR 使用相同的颜色空间，只是颜色顺序相反。

请注意，在 `scikit-image` 中，我们通常指的是 `rows` 和 `columns`，而不是宽度和高度(see Coordinate conventions)。

对于沿最后一个轴具有颜色的图像，以下指令有效地反转颜色的顺序，使行和列不受影响。

In [ ]:
image = image[:, :, ::-1]

### 5.3.2. Using an image from OpenCV with `skimage`

如果 cv_image 是无符号字节数组，`skimage` 默认会理解它。如果您喜欢使用浮点图像，可以使用 `img_as_float()` 来转换图像：

In [ ]:
from skimage.util import img_as_float
image = img_as_float(any_opencv_image)

### 5.3.3. Using an image from `skimage` with OpenCV

可以使用 `img_as_ubyte()` 实现相反的效果：

In [ ]:
from skimage.util import img_as_ubyte
cv_image = img_as_ubyte(any_skimage_image)

## 5.4. Image processing pipeline

这种数据类型行为允许您将任何 `skimage` 函数串在一起，而无需担心图像数据类型。另一方面，如果您想使用需要特定数据类型的自定义函数，则应调用其中一个数据类型转换函数（此处，`func1` 和 `func2` 是 `skimage` 函数）：

In [ ]:
from skimage.util import img_as_float
image = img_as_float(func1(func2(image)))
processed_image = custom_func(image)

更好的是，您可以在内部转换图像并使用简化的处理管道：

In [ ]:
def custom_func(image):
    image = img_as_float(image)
    # do something
    processed_image = custom_func(func1(func2(image)))

## 5.5. Rescaling intensity values

如果可能，函数应避免盲目拉伸图像强度（例如，重新缩放浮点图像，使最小和最大强度分别为 0 和 1），因为这会严重扭曲图像。例如，如果您要在黑暗图像中寻找明亮的标记，则可能存在不存在标记的图像；将其输入强度扩展到整个范围将使背景噪声看起来像标记。

然而，有时您的图像应该跨越整个强度范围，但实际上却没有。例如，某些相机存储每像素 10 位、12 位或 14 位深度的图像。如果这些图像存储在 dtype uint16 的数组中，则图像将不会扩展到整个强度范围，因此会显得比应有的更暗。要纠正此问题，您可以使用 `rescale_intensity` 函数重新缩放图像，以便它使用完整的 dtype 范围：

In [ ]:
from skimage import exposure
image = exposure.rescale_intensity(img10bit, in_range=(0, 2**10 - 1))

此处，`in_range` 参数设置为 10 位图像的最大范围。默认情况下，`rescale_intensity` 会拉伸 `in_range` 的值以匹配 dtype 的范围。 `rescale_intensity` 还接受字符串作为 `in_range` 和 `out_range` 的输入，因此上面的示例也可以写为：

In [ ]:
image = exposure.rescale_intensity(img10bit, in_range='uint10')

## 5.6. Note about negative values

人们经常用带符号的数据类型表示图像，即使他们只操纵图像的正值（例如，在 int8 图像中仅使用 0-127）。因此，转换函数仅将有符号数据类型的正值分布在无符号数据类型的整个范围内。换句话说，当从有符号数据类型转换为无符号数据类型时，负值将被截断为 0。（在带符号的数据类型之间转换时，会保留负值。）为了防止这种剪切行为，您应该事先重新缩放图像：

In [ ]:
image = exposure.rescale_intensity(img_int32, out_range=(0, 2**31 - 1))
img_uint8 = img_as_ubyte(image)

此行为是对称的：无符号数据类型中的值仅分布在有符号数据类型的正范围内。